In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8,hu;q=0.7",
    "Referer": "https://google.com",
    "DNT": "1"
}

In [3]:
url = 'https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc'
response = requests.get(url, headers=headers)

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
movie_containers = soup.find_all('div', class_='lister-item-content')

In [5]:
# extract movie title and URL
top_20_urls = []
top_20_names = []
for movie in movie_containers [0:5]:
    header = movie.find('h3', class_='lister-item-header')
    title = header.a.text
    movie_url = 'https://www.imdb.com' + header.a['href']
    top_20_urls.append(movie_url)
    top_20_names.append(title)
#top_20_names

In [6]:
# extract ratings
top_20_rating = []
for movie in movie_containers [0:5]:
    rating = float(movie.find('div', class_='inline-block ratings-imdb-rating').text.strip())
    top_20_rating.append(rating)
#top_20_rating

In [7]:
# extract number of raters
top_20_rater = []
for movie in movie_containers [0:5]:
    num_raters = movie.find('span', attrs={'name': 'nv'}).text
    num_raters = int(num_raters.replace(',', ''))
    top_20_rater.append(num_raters)
#top_20_rater

In [8]:
def get_num_oscars(top_20_urls):
    num_oscars = []

    for url in top_20_urls:
        # Format the URL to include the "awards" path
        award_url = url.replace("?ref_=adv_li_tt", "awards")

        # Get the HTML content of the URL
        response = requests.get(award_url, headers=headers)
        html_content = response.text

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the award items with category "Oscar" and outcome "Winner"
        oscar_winners = soup.find_all('td', class_='title_award_outcome')
        oscar_winners = [ow for ow in oscar_winners if ow.find('span', class_='award_category').text == 'Oscar' and ow.find('b').text == 'Winner']

        # Get the number of Oscars won by the movie
        if len(oscar_winners) > 0:
            num_oscar = int(oscar_winners[0]['rowspan'])
        else:
            num_oscar = 0

        num_oscars.append(num_oscar)

    return num_oscars


In [9]:
top_20_oscar = []
for movie_url in top_20_urls:
    num_oscars = get_num_oscars(movie_url)
    top_20_oscar.append(num_oscars)


MissingSchema: Invalid URL 'h': No scheme supplied. Perhaps you meant http://h?

In [ ]:
top_20_oscar

In [ ]:
top_20_urls

In [ ]:
get_num_oscars('https://www.imdb.com/title/tt0167260/?ref_=adv_li_tt')

In [ ]:
top_20_oscar = []
oscar_points = []
for oscar in top_20_oscar [0:5]:
    if oscar == 1 or oscar == 2:
        oscar_points.append(0.3)
    elif oscar == 3 or oscar == 4 or oscar == 5:
        oscar_points.append(0.5)
    elif oscar >= 6 and oscar <= 10:
        oscar_points.append(1)
    elif oscar > 10:
        oscar_points.append(1.5)
    else:
        oscar_points.append(0)

In [ ]:
oscar_points

In [ ]:
# Find highest number of raters
max_raters = max(top_20_rater)

# Calculate 0.1 point deduction for every 100,000 difference in raters from the maximum
point_deduction = 0.1 * (top_20_rater - max_raters) / 100000

# Round point deduction to one decimal point
point_deduction = [round(p, 1) for p in point_deduction]

# Add new points column to table
table.append(new_points)

In [ ]:
# Combine the data into a single table
data = {'url': top_20_urls, 'name': top_20_names, 'rating': top_20_rating, 'rater': top_20_rater, 'oscars': top_20_oscar}
df = pd.DataFrame(data)

# Create an Excel writer
writer = pd.ExcelWriter('Datapao_IMDB_scraper.xlsx')

# Write the data to a sheet
df.to_excel(writer, index=False, sheet_name='Top Movies')

# Save the file
writer.save()
